In [108]:
import pandas as pd
import graphlab as gl
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline

In [266]:
df=pd.read_csv('data/CAX_Bidding_TRAIN_Molecule_3_4_5.csv')

In [267]:
col=['Molecule','Winning_price_per_standard_unit','Region','Length_of_contract_in_Months', 'Presentation', 'Start_date']


In [268]:
df2=df[col]

In [269]:
df2=df2.dropna()
df2.head()

,Molecule,Winning_price_per_standard_unit,Region,Length_of_contract_in_Months,Presentation,Start_date
0,Molecule3,1.56353,Region6,36,Presentation8,01-01-2009
1,Molecule3,1.56353,Region6,20,Presentation8,01-05-2010
2,Molecule3,1.77000,Region13,24,Presentation8,01-05-2010
3,Molecule3,1.77000,Region9,24,Presentation8,01-07-2010
4,Molecule3,1.50000,Region7,12,Presentation8,01-08-2010


In [270]:
df2['Start_date']=pd.to_datetime(df2['Start_date'])
def days_since_loe(a, b):
    if a=='Molecule3':
        return (b - datetime(2009,1,1,0,0,0)).days
    elif a=='Molecule4':
        return (b - datetime(2011,1,1,0,0,0)).days
    else:
        return (b - datetime(2011,1,1,0,0,0)).days


In [204]:
#df2

In [271]:
df2['Value'] = df2.apply(lambda row: days_since_loe(row['Molecule'], row['Start_date']), axis=1)

In [272]:
df2=df2[df2['Value']>0]

In [273]:
df2.head()

,Molecule,Winning_price_per_standard_unit,Region,Length_of_contract_in_Months,Presentation,Start_date,Value
1,Molecule3,1.56353,Region6,20,Presentation8,2010-01-05,369
2,Molecule3,1.77000,Region13,24,Presentation8,2010-01-05,369
3,Molecule3,1.77000,Region9,24,Presentation8,2010-01-07,371
4,Molecule3,1.50000,Region7,12,Presentation8,2010-01-08,372
5,Molecule3,1.45000,Region1,36,Presentation8,2010-01-11,375


In [274]:
pList = df['Presentation'].unique()
pDict = {}
for x in pList:
    for index, row in df.iterrows():
        pDict[x] = df[df['Presentation']== x]['Winning_price_per_standard_unit'].max()
        break
            

In [275]:
#df10 = df[(df['Molecule'] == 'Molecule3') & (df['Region'] == 'Region6')]
#df10 = df10.sort(columns='Start_date')
#df10[['Start_date', 'Winning_price_per_standard_unit', 'Presentation']]
pDict

{'Presentation10': 1.89,
 'Presentation11': 5.659,
 'Presentation12': 24.87,
 'Presentation13': 1.54,
 'Presentation14': 2.71084,
 'Presentation15': 3.04107,
 'Presentation16': 1.52035,
 'Presentation17': 3.04107,
 'Presentation18': 0.76035,
 'Presentation19': 1.52035,
 'Presentation8': 4.87,
 'Presentation9': 11.2694}

In [276]:
pDict['Presentation10']
def factor_price(price, presentation):
    return price/pDict[presentation]

In [277]:
df2['Winning_price_per_standard_unit'] = df2.apply(lambda row: factor_price(row['Winning_price_per_standard_unit'], row['Presentation']), axis=1)

In [279]:
df2.head()

,Molecule,Winning_price_per_standard_unit,Region,Length_of_contract_in_Months,Presentation,Start_date,Value
1,Molecule3,0.321053,Region6,20,Presentation8,2010-01-05,369
2,Molecule3,0.363450,Region13,24,Presentation8,2010-01-05,369
3,Molecule3,0.363450,Region9,24,Presentation8,2010-01-07,371
4,Molecule3,0.308008,Region7,12,Presentation8,2010-01-08,372
5,Molecule3,0.297741,Region1,36,Presentation8,2010-01-11,375


In [280]:
df2=df2.drop(['Molecule', 'Presentation', 'Start_date'],1)

In [281]:
df2.head()

,Winning_price_per_standard_unit,Region,Length_of_contract_in_Months,Value
1,0.321053,Region6,20,369
2,0.363450,Region13,24,369
3,0.363450,Region9,24,371
4,0.308008,Region7,12,372
5,0.297741,Region1,36,375


In [282]:
df2=pd.get_dummies(df2)

In [283]:
y=df2.as_matrix()[:,0]

In [284]:
y.shape

(662,)

In [158]:
#df2=df2.drop('Winning_price_per_standard_unit',axis=1)

In [285]:
X=df2.as_matrix()[:,1:]

In [286]:
X.shape

(662, 22)

In [287]:
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [293]:
randForest=RandomForestRegressor()

In [300]:
#randForest.fit_transform(X_train,y_train)

In [295]:
randForest.score(X_test,y_test)

0.67958866670495044

In [296]:
y_rand_pred=randForest.predict(X_test)

In [297]:
mean_squared_error(y_test,y_rand_pred)

0.026641934617598406

In [288]:
reg=linear_model.LinearRegression()

In [289]:
reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [322]:
reg.score(X_test,y_test)

0.23788988472130912

In [291]:
y_pred=reg.predict(X_test)

In [307]:
y_pred[0:10]

array([ 0.06600635,  0.22590543,  0.25396411,  0.35657539,  0.34605634,
        0.02895916,  0.15389767,  0.17951893,  0.21899295,  0.47066038])

In [308]:
y_test[0:10]

array([ 0.00690218,  0.05714935,  0.0394598 ,  0.74183186,  0.72097544,
        0.03246753,  0.05136975,  0.02565199,  0.02251709,  0.72408469])

In [83]:
#plt.scatter(y_pred,y_test)

In [292]:
mean_squared_error(y_test,y_pred)

0.06336881924200953

In [316]:
df_test=pd.read_csv('data/CAX_Bidding_TEST_Molecule_6_Post_LOE.csv')

In [318]:
#df_test.head()

In [107]:
col_test=['Region','Length_of_contract_in_Months','']

In [309]:
df1_test=df_test[col]

In [310]:
df1_test.head()

,Molecule,Winning_price_per_standard_unit,Region,Length_of_contract_in_Months,Presentation,Start_date
0,Molecule6,NaN,Region2,4,Presentation20,01-03-2011
1,Molecule6,NaN,Region3,36,Presentation20,01-03-2011
2,Molecule6,NaN,Region20,15,Presentation20,22-04-2011
3,Molecule6,NaN,Region2,8,Presentation20,01-05-2011
4,Molecule6,NaN,Region9,10,Presentation20,01-05-2011


In [314]:
df1_test=df1_test.drop(['Winning_price_per_standard_unit','Presentation','Molecule'],1)

In [315]:
df1_test.head()

,Region,Length_of_contract_in_Months,Start_date
0,Region2,4,01-03-2011
1,Region3,36,01-03-2011
2,Region20,15,22-04-2011
3,Region2,8,01-05-2011
4,Region9,10,01-05-2011


In [110]:
df2_test=pd.get_dummies(df1_test)

In [111]:
df2_test['Value'] = df2_test.apply(lambda row: days_since_loe(row['Molecule'], row['Start_date']), axis=1)

,Length_of_contract_in_Months,Region_Region1,Region_Region10,Region_Region11,Region_Region12,Region_Region13,Region_Region14,Region_Region15,Region_Region16,Region_Region17,...,Region_Region19,Region_Region2,Region_Region20,Region_Region3,Region_Region4,Region_Region5,Region_Region6,Region_Region7,Region_Region8,Region_Region9
0,4,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,36,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,15,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [112]:
x_ttest=df2_test.as_matrix()

In [113]:
x_ttest.shape

(152, 21)

In [114]:
ytest_pred=reg.predict(x_ttest)

In [121]:
type(ytest_pred)

numpy.ndarray

In [122]:
import csv
b = open('test.csv', 'w')
a = csv.writer(b)
data = ytest_pred
a.writerow(data)
b.close()

In [319]:
df_test=pd.read_csv('data/CAX_Bidding_TEST_Molecule_6_Post_LOE.csv')

col_test =['Molecule', 'Region','Length_of_contract_in_Months', 'Presentation', 'Start_date']

df2_test =df_test[col_test]

df2_test.head()

df2_test['Start_date']=pd.to_datetime(df2_test['Start_date'])
def days_since_loe(a, b):
    if a=='Molecule6':
        return (b - datetime(2011,1,1,0,0,0)).days
    else:
        return (b - datetime(2011,1,1,0,0,0)).days

df2_test['Value'] = df2_test.apply(lambda row: days_since_loe(row['Molecule'], row['Start_date']), axis=1)

df2_test=df2_test[df2_test['Value']>0]

#df2_test

df_pres=pd.read_csv('data/CAX_Bidding_TEST_Molecule_6_Pre_LOE.csv')

pList = df_pres['Presentation'].unique()
pDict = {}
for x in pList:
    for index, row in df_pres.iterrows():
        pDict[x] = df_pres[df_pres['Presentation']== x]['Winning_price_per_standard_unit'].max()
        break
            

pDict

def factor_price(price, presentation):
    return price/pDict[presentation]

df2_test.head()

df3_test = df2_test.drop(['Molecule', 'Presentation', 'Start_date'], 1)

df3_test.head()



/Users/ssing25/anaconda2/envs/dato-env/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ssing25/anaconda2/envs/dato-env/lib/python2.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Region,Length_of_contract_in_Months,Value
0,Region2,4,2
1,Region3,36,2
2,Region20,15,111
3,Region2,8,4
4,Region9,10,4


In [320]:
df3_test=pd.get_dummies(df3_test)

df3_test.head()

,Length_of_contract_in_Months,Value,Region_Region1,Region_Region10,Region_Region11,Region_Region12,Region_Region13,Region_Region14,Region_Region15,Region_Region16,...,Region_Region19,Region_Region2,Region_Region20,Region_Region3,Region_Region4,Region_Region5,Region_Region6,Region_Region7,Region_Region8,Region_Region9
0,4,2,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,36,2,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,15,111,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,8,4,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,10,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [321]:
x_test_mtx=df3_test.as_matrix()

In [323]:
y_test_pred=reg.predict(x_test_mtx)

In [329]:
y_test_pred.shape

(152,)

In [325]:
df2_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 0 to 151
Data columns (total 6 columns):
Molecule                        152 non-null object
Region                          152 non-null object
Length_of_contract_in_Months    152 non-null int64
Presentation                    152 non-null object
Start_date                      152 non-null datetime64[ns]
Value                           152 non-null int64
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 8.3+ KB


In [327]:
y_test.shape

(166,)

In [339]:
df2_test.loc[1,'Presentation']

'Presentation20'

In [348]:
y_final = []
for i,j in enumerate(y_test_pred):
    y_final.append(j * pDict[df2_test.loc[i,'Presentation']])

In [350]:
#y_final

In [ ]:
y_final_pred = np.asarray([])

In [351]:
y_rand_predict=randForest.predict(x_test_mtx)

In [353]:
y_rand_final = []
for i,j in enumerate(y_rand_predict):
    y_rand_final.append(j * pDict[df2_test.loc[i,'Presentation']])

In [354]:
y_rand_final

[19.414207999488763,
 12.312638593668648,
 17.528536881182259,
 17.989659597737859,
 18.181500813926093,
 18.878015708107622,
 19.413982518714178,
 15.169029981593029,
 16.49895779236568,
 16.275154096510253,
 18.673706992254427,
 17.989659597737859,
 16.485553175705761,
 18.510642989173753,
 14.326289351905119,
 15.022843478552664,
 12.286915229596083,
 13.915382685644849,
 14.543089840358359,
 10.258487246300854,
 15.60397805803219,
 12.670007383512029,
 15.58854777177906,
 12.594459154697169,
 11.979833597793769,
 11.979833597793769,
 0.84183888414369723,
 0.58927837461578991,
 1.8438969587630925,
 2.7818236739311391,
 1.0730108185033493,
 6.1425023404190853,
 1.2358324242818519,
 0.95607389238294149,
 0.95607389238294149,
 1.7595173271465785,
 0.74142599821342259,
 1.5657381956242968,
 0.56689075803785627,
 0.71771502988514402,
 0.63178677268717465,
 0.52361413446632321,
 0.43459644468869102,
 0.37411153921226692,
 0.51088274765427388,
 0.56615005268737617,
 0.88228958969361104,
 0